# RouterChain多链路由组合

## 源码分析

### 类定义和属性

In [ ]:
class RouterChain(Chain, ABC):
    """Chain that outputs the name of a destination chain and the inputs to it."""

    @property
    def output_keys(self) -> List[str]:
        return ["destination", "next_inputs"]

- `RouterChain`继承自`Chain`并实现了`ABC`(Abstract Base Class)，表明这是一个抽象类，不能直接实例化。
- `output_keys`: 这是一个属性方法，返回一个包含两个字符串的列表：`"destination"` 和 `"next_inputs"`。它定义了该链的输出键：
  - `"destination"`：目标链的名称。
  - `"next_inputs"`：要传递给目标链的输入。

### `route` 方法

In [ ]:
[docs] def route(self, inputs: Dict[str, Any], callbacks: Callbacks = None) -> Route:
    """
    Route inputs to a destination chain.

    Args:
        inputs: inputs to the chain
        callbacks: callbacks to use for the chain

    Returns:
        a Route object
    """
    result = self(inputs, callbacks=callbacks)
    return Route(result["destination"], result["next_inputs"])

- `route`: 这是一个同步方法，用于根据输入数据确定目标链，并传递相应的输入数据。具体操作如下：
  - `inputs`: 传入的输入数据，通常是一个字典。
  - `callbacks`: 可选的回调函数，用于链执行期间的回调操作。
  - `result`: 调用 `self(inputs, callbacks=callbacks)`，执行当前链并获取结果。结果应该是一个字典，包含 `destination` 和 `next_inputs` 两个键。
  - `Route`: 返回一个 `Route` 对象，包含目标链的名称和传递给目标链的输入数据。

### `aroute`方法

In [ ]:
[docs] async def aroute(
    self, inputs: Dict[str, Any], callbacks: Callbacks = None
) -> Route:
    result = await self.acall(inputs, callbacks=callbacks)
    return Route(result["destination"], result["next_inputs"])

- `aroute`: 这是 route 方法的异步版本，功能相同，但支持异步操作。
  - `inputs`: 传入的输入数据，通常是一个字典。
  - `callbacks`: 可选的回调函数，用于链执行期间的回调操作。
  - `result`: 使用 await self.acall(inputs, callbacks=callbacks) 执行当前链的异步调用，获取结果。
  - `Route`: 返回一个 Route 对象，包含目标链的名称和传递给目标链的输入数据。

### 总结
`RouterChain` 是一个抽象类，提供了路由输入数据到目标链的逻辑。它通过 `route` 和 `aroute` 方法根据输入数据确定目标链，并生成要传递给目标链的输入。这使得 `RouterChain` 能够根据具体条件灵活地选择不同的链进行处理，适用于复杂流程控制场景。

## demo

In [41]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.base import RouterChain
from langchain_community.chat_models import ChatTongyi
from langchain.callbacks.base import Callbacks
import os

In [42]:
# 从环境变量中获取API密钥，用于初始化 ChatTongyi 模型
api_key = os.getenv("KEY_TONGYI")
if not api_key:
    raise ValueError("API Key is not set. Please ensure that the 'KEY_TONGYI' environment variable is set.")

# 初始化 ChatTongyi 模型，设置文本生成的温度参数，温度越低生成的文本越接近输入
llm = ChatTongyi(
    dashscope_api_key=api_key,
    temperature=0,  # 设置生成文本的倾向，值越小生成的文本越接近输入
    streaming=True
)

In [43]:
# 定义第一个chain
# 提示词模板
task_description_template = PromptTemplate(
  input_variables=["input"],
  template="根据{input}这个主题写一首诗。",
)
generate_poem_chain = LLMChain(llm=llm, prompt=task_description_template, output_key="poem_content")

In [91]:
# 定义第二个chain
# 提示词模板
subtask_description_template_eval = PromptTemplate(
  input_variables=["input"],
  template="根据{input}这首诗,写一个评价。",
)
evaluate_poem_chain = LLMChain(llm=llm, prompt=subtask_description_template_eval, output_key="output")

In [92]:
# 定义第三个chain
# 提示词模板
subtask_description_template_trans = PromptTemplate(
  input_variables=["input"],
  template="用优美的英语翻译{input}这首诗",
)
trans_poem_chain = LLMChain(llm=llm, prompt=subtask_description_template_trans, output_key="output")

In [93]:
# 创建一个简单的路由链
from gc import callbacks
from typing import Any, Dict, List

# 创建一个简单的路由链
class SimpleRouterChain(RouterChain):
    """A simple router chain example."""

    @property
    def input_keys(self) -> List[str]:
        """Return expected input keys."""
        return ["input"]

    def _call(self, inputs: Dict[str, Any], callbacks: Callbacks = None) -> Dict[str, str]:
        # 路由逻辑：如果输入包含 "A"，选择 ChainA，否则选择 ChainB
        if "评价" in inputs["input"]:
            print("评价")
            return {"destination": "evaluate_poem_chain", "next_inputs": {"input": inputs["input"]}}
        
        elif "翻译" in inputs["input"]:
            print("翻译")
            return {"destination": "trans_poem_chain", "next_inputs": {"input": inputs["input"]}}

        else: 
            print("无")
            return None
        
    async def _acall(self, inputs: Dict[str, Any], callbacks: Callbacks = None) -> Dict[str, str]:
        # 异步路由逻辑
        if "评价" in inputs["input"]:
            print("评价")
            return {"destination": "evaluate_poem_chain", "next_inputs": {"input": inputs["input"]}}
        
        elif "翻译" in inputs["input"]:
            print("翻译")
            return {"destination": "trans_poem_chain", "next_inputs": {"input": inputs["input"]}}

        else: 
            print("无")
            return None

In [94]:
# 初始化路由链
router_chain = SimpleRouterChain()

In [95]:
# 定义一个简单的函数来模拟目标链的选择
def execute_chain(router_chain, input_text):
    route = router_chain.route({"input": input_text})
    print(route)
    if route.destination == "evaluate_poem_chain":
        return evaluate_poem_chain.run(route.next_inputs)
    elif route.destination == "trans_poem_chain":
        return trans_poem_chain.run(route.next_inputs)
    else:
        return "No matching chain found."

In [27]:
# 生成一首诗
question = '女朋友生日快乐'
resopne_poem = generate_poem_chain.invoke(question)

In [28]:
# 评价的路由问题
question2evaluate = "评价这首诗："+resopne_poem['poem_content']
question2evaluate

'评价这首诗：【生日快乐，我的挚爱】\n\n在春日的温柔里，你如晨曦初照，\n轻轻唤醒沉睡的心房，让爱意绽放。\n你的笑靥，是夏日里最清新的凉风，\n吹散了生活的尘埃，留下一片宁静。\n\n秋月之下，你是我手中那盏明灯，\n照亮前行的道路，驱走夜的寂寞与冷。\n冬雪皑皑，你温暖如春，融化我心中的冰封，\n让我相信，即使世界寒冷，爱永不止息。\n\n你是我生命中最美的风景，\n每一次相遇，都是上天赐予的幸运。\n在这特别的日子里，我想对你说：\n生日快乐，愿你的每一天都如诗如画，充满欢声笑语。\n\n愿你的梦想如同繁星，璀璨夺目，\n每一步前行，都能收获属于你的幸福。\n无论风雨，无论晴空，都有我陪伴在侧，\n守护你，直到地老天荒，海枯石烂。\n\n亲爱的，让我们手牵手，共赴未来的旅程，\n在彼此的爱中，找到永恒的归宿。\n在这个特别的日子，我要大声告诉你：\n我爱你，比昨天多一点，比明天少一点，但永远不变。\n\n愿这诗，如一束光，照亮你的生日，\n愿这份心意，如春风，温暖你的心房。\n生日快乐，我的挚爱，愿你的世界，\n永远充满爱，充满希望，充满欢笑。'

In [29]:
# 翻译的路由问题
question2trans = "翻译这首诗："+resopne_poem['poem_content']
question2trans

'翻译这首诗：【生日快乐，我的挚爱】\n\n在春日的温柔里，你如晨曦初照，\n轻轻唤醒沉睡的心房，让爱意绽放。\n你的笑靥，是夏日里最清新的凉风，\n吹散了生活的尘埃，留下一片宁静。\n\n秋月之下，你是我手中那盏明灯，\n照亮前行的道路，驱走夜的寂寞与冷。\n冬雪皑皑，你温暖如春，融化我心中的冰封，\n让我相信，即使世界寒冷，爱永不止息。\n\n你是我生命中最美的风景，\n每一次相遇，都是上天赐予的幸运。\n在这特别的日子里，我想对你说：\n生日快乐，愿你的每一天都如诗如画，充满欢声笑语。\n\n愿你的梦想如同繁星，璀璨夺目，\n每一步前行，都能收获属于你的幸福。\n无论风雨，无论晴空，都有我陪伴在侧，\n守护你，直到地老天荒，海枯石烂。\n\n亲爱的，让我们手牵手，共赴未来的旅程，\n在彼此的爱中，找到永恒的归宿。\n在这个特别的日子，我要大声告诉你：\n我爱你，比昨天多一点，比明天少一点，但永远不变。\n\n愿这诗，如一束光，照亮你的生日，\n愿这份心意，如春风，温暖你的心房。\n生日快乐，我的挚爱，愿你的世界，\n永远充满爱，充满希望，充满欢笑。'

In [97]:
output_evaluate = execute_chain(router_chain, question2evaluate)

评价
Route(destination='evaluate_poem_chain', next_inputs={'input': '评价这首诗：【生日快乐，我的挚爱】\n\n在春日的温柔里，你如晨曦初照，\n轻轻唤醒沉睡的心房，让爱意绽放。\n你的笑靥，是夏日里最清新的凉风，\n吹散了生活的尘埃，留下一片宁静。\n\n秋月之下，你是我手中那盏明灯，\n照亮前行的道路，驱走夜的寂寞与冷。\n冬雪皑皑，你温暖如春，融化我心中的冰封，\n让我相信，即使世界寒冷，爱永不止息。\n\n你是我生命中最美的风景，\n每一次相遇，都是上天赐予的幸运。\n在这特别的日子里，我想对你说：\n生日快乐，愿你的每一天都如诗如画，充满欢声笑语。\n\n愿你的梦想如同繁星，璀璨夺目，\n每一步前行，都能收获属于你的幸福。\n无论风雨，无论晴空，都有我陪伴在侧，\n守护你，直到地老天荒，海枯石烂。\n\n亲爱的，让我们手牵手，共赴未来的旅程，\n在彼此的爱中，找到永恒的归宿。\n在这个特别的日子，我要大声告诉你：\n我爱你，比昨天多一点，比明天少一点，但永远不变。\n\n愿这诗，如一束光，照亮你的生日，\n愿这份心意，如春风，温暖你的心房。\n生日快乐，我的挚爱，愿你的世界，\n永远充满爱，充满希望，充满欢笑。'})


In [98]:
output_evaluate

'这首诗充满了深情与诗意，作者以四季为线索，巧妙地将爱情的美好与复杂情感描绘得淋漓尽致。从春日的温柔、夏日的清新、秋月的指引到冬雪的温暖，每一个季节的意象都寓意着爱情的不同阶段和特质，既展现了爱情的甜蜜与温馨，也触及到了其深度与广度。诗中的比喻和象征运用得恰到好处，不仅增强了语言的表现力，也让读者能够更加深刻地感受到诗人对挚爱的深情厚谊。\n\n诗的语言优美而富有感染力，每一句都充满了对对方的赞美和承诺，无论是对对方梦想的支持、对未来旅程的共同期待，还是对永恒爱情的坚信，都透露出一种坚定而美好的情感。这种情感的表达方式既浪漫又真诚，让人感到温暖和感动。\n\n此外，诗中还融入了一些哲学思考，如“无论风雨，无论晴空，都有我陪伴在侧”，这不仅是对当前关系的承诺，也暗示了一种对生活态度的积极理解——即使面对困难和挑战，有爱相伴，就能找到前进的力量。\n\n总的来说，这首诗是一首充满深情、诗意与哲理的作品，它不仅表达了对挚爱的深厚情感，也传递了对美好生活的向往和对未来的乐观态度。通过四季的变换，诗人成功地构建了一个既具象又抽象的爱情世界，使得这首诗具有了广泛的共鸣性和艺术感染力。'

In [99]:
output_trans = execute_chain(router_chain, question2trans)

翻译
Route(destination='trans_poem_chain', next_inputs={'input': '翻译这首诗：【生日快乐，我的挚爱】\n\n在春日的温柔里，你如晨曦初照，\n轻轻唤醒沉睡的心房，让爱意绽放。\n你的笑靥，是夏日里最清新的凉风，\n吹散了生活的尘埃，留下一片宁静。\n\n秋月之下，你是我手中那盏明灯，\n照亮前行的道路，驱走夜的寂寞与冷。\n冬雪皑皑，你温暖如春，融化我心中的冰封，\n让我相信，即使世界寒冷，爱永不止息。\n\n你是我生命中最美的风景，\n每一次相遇，都是上天赐予的幸运。\n在这特别的日子里，我想对你说：\n生日快乐，愿你的每一天都如诗如画，充满欢声笑语。\n\n愿你的梦想如同繁星，璀璨夺目，\n每一步前行，都能收获属于你的幸福。\n无论风雨，无论晴空，都有我陪伴在侧，\n守护你，直到地老天荒，海枯石烂。\n\n亲爱的，让我们手牵手，共赴未来的旅程，\n在彼此的爱中，找到永恒的归宿。\n在这个特别的日子，我要大声告诉你：\n我爱你，比昨天多一点，比明天少一点，但永远不变。\n\n愿这诗，如一束光，照亮你的生日，\n愿这份心意，如春风，温暖你的心房。\n生日快乐，我的挚爱，愿你的世界，\n永远充满爱，充满希望，充满欢笑。'})


In [100]:
output_trans

"In the gentle embrace of spring, you shine like the dawn,\nGently awakening the slumbering heart, letting love bloom.\nYour smile is the freshest breeze of summer,\nBlowing away life's dust, leaving tranquility behind.\n\nUnder the moon of autumn, you are my guiding light,\nIlluminating the path ahead, driving away loneliness and chill of night.\nIn winter's snow, you are warm as spring, melting the ice within my heart,\nLetting me believe that even if the world is cold, love never fades.\n\nYou are the most beautiful scenery in my life,\nEvery encounter is a blessing bestowed by fate.\nOn this special day, I want to say to you:\nHappy birthday, may each day be as poetic as a painting, filled with laughter and joy.\n\nMay your dreams be as brilliant as stars, dazzling and bright,\nEvery step forward brings you the happiness that belongs to you.\nNo matter the storm or clear sky, I will always be by your side,\nGuarding you until eternity, through time and forever.\n\nMy dear, let us h